In [ ]:
import yfinance as yf
import pandas as pd
import requests
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint, adfuller
from datetime import datetime

In [ ]:
def download_yfinance_data(tickers, start_date, end_date):
    """
    Download historical stock data from Yahoo Finance for the given tickers and date range.
    Combine all data into a single DataFrame with a fixed daily index.
    """
    date_dataframe = pd.DataFrame(pd.date_range(start=start_date, end=end_date, freq='D'), columns=['Date'])

    for ticker in tqdm(tickers):
        try:
            data_ticker = yf.download(ticker, start=start_date, end=end_date, multi_level_index=False)
            if not data_ticker.empty:
                # Determine which column to use
                close_column = 'Adj Close' if 'Adj Close' in data_ticker.columns else 'Close'
                data_extract = data_ticker[[close_column]].rename(columns={close_column: ticker})
                date_dataframe = date_dataframe.merge(data_extract, left_on='Date', right_index=True, how='left')
        except Exception as e:
            print(f"Error for {ticker}: {e}")

    return date_dataframe

In [ ]:
def get_all_tickers():
    # API URL
    url = "https://api.nasdaq.com/api/screener/stocks?tableonly=true&download=true"
    # Headers (User-Agent is required to avoid access restrictions)
    headers = {"User-Agent": "Mozilla/5.0"}

    # Fetch data from the API
    response = requests.get(url, headers=headers)
    data = response.json()

    # Extract relevant rows
    rows = data.get("data", {}).get("rows", [])

    # Create DataFrame
    df_result = pd.DataFrame(rows, columns=["symbol", "name", "country", "ipoyear", "industry"])

    # Display the first few rows
    return df_result

In [ ]:
# df = pd.read_csv("Data/yfinance_data.csv")

In [ ]:
# df_all_tickers = get_all_tickers() #6898

In [ ]:
# start_date = "2024-01-01"
# end_date = "2024-12-31"
# tickers = df_all_tickers['symbol'].to_list()  # Example tickers
# # tickers = ['AAPL','MSFT','GOOGL']
#
# df = download_yfinance_data(tickers, start_date, end_date)
#
# if df is not None:
#     df.to_csv("Data/yfinance_data.csv")
#     print("Data successfully downloaded and saved to yfinance_data.csv")

In [ ]:
# df = df.set_index('Date')
# df_clean = df.dropna(axis=1, thresh=int(len(df.index)/2), inplace=False)
# # df_filled = df_clean.interpolate(method='linear')
# # c = df_filled.corr().abs()
# c = df_clean.corr().abs()
# # Remove self-correlation and keep only one side of the matrix
# c = c.where(~np.tril(np.ones(c.shape), k=0).astype(bool))
# s = c.unstack()
# so = s.sort_values(kind="quicksort", ascending=False).reset_index()
# so.columns = ['ticker_1', 'ticker_2', 'correlation']
# so.to_csv('ticker_correlations.zip')

In [ ]:
# so[so['correlation']<1]


In [ ]:

wiki_data=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies') # Open the link and download S&P company details in a table
data = wiki_data[0] # All data is stored in first cell
sorted_data = data.sort_values(by=['Symbol'], ascending=True) # Sort the dataframe on ticker in alphabetical ascending order
# Convert the dataframe to csv file
sorted_data.to_csv('Data/S&P500Tickers.csv', mode='w', index=False) #index is False as we don't want to write index in csv file
sorted_data

In [ ]:
# Analysis on snp500 only
start_date = "2020-01-01"
end_date = "2025-01-31"
tickers = sorted_data['Symbol'].to_list()  # Example tickers

df = download_yfinance_data(tickers, start_date, end_date)

if df is not None:
    df.to_csv("Data/yfinance_snp500_data_202001_202501.csv")
    print("Data successfully downloaded and saved to yfinance_data.csv")

In [ ]:
df = df.set_index('Date')

In [ ]:
# df = df.set_index('Date')
df_clean = df.dropna(axis=1, thresh=int(len(df.index)/2), inplace=False)
# df_filled = df_clean.interpolate(method='linear')
# c = df_filled.corr().abs()
c = df_clean.corr().abs()
# Remove self-correlation and keep only one side of the matrix
c = c.where(~np.tril(np.ones(c.shape), k=0).astype(bool))
s = c.unstack()
so = s.sort_values(kind="quicksort", ascending=False).reset_index()
so.columns = ['ticker_1', 'ticker_2', 'correlation']
so.to_csv('Data/snp500_correlations_202001_202501.zip')

In [ ]:
# read csv files
# df = pd.read_csv("yfinance_snp500_data_202001_202501.csv")
# so = pd.read_csv('snp500_correlations_202001_202501.zip')

In [ ]:
so[so['correlation']<1]

In [ ]:
## check for cointegration
def check_stationarity(series):
    """Returns the p-value from the Augmented Dickey-Fuller (ADF) test."""
    return adfuller(series)[1]  # ADF test p-value

def calculate_cointegration(corr_df, timeseries_df):
    """
    Takes in:
    - corr_df: DataFrame with columns ['ticker1', 'ticker2', 'correl']
    - timeseries_df: DataFrame where each column is a ticker's time series.

    Adds three new columns:
    - 'p1_stationary': ADF p-value for ticker1
    - 'p2_stationary': ADF p-value for ticker2
    - 'p_cointegration': Cointegration test p-value
    """

    results = []  # Store results for efficient DataFrame construction

    for index, row in corr_df.iterrows():
        t1, t2 = row["ticker_1"], row["ticker_2"]

        # Get time series data
        series1 = timeseries_df[t1].dropna()
        series2 = timeseries_df[t2].dropna()

        # Ensure both series have the same length
        min_len = min(len(series1), len(series2))
        series1, series2 = series1.iloc[-min_len:], series2.iloc[-min_len:]

        # Check stationarity (ADF test)
        p1 = check_stationarity(series1)
        p2 = check_stationarity(series2)

        # Check cointegration only if both series are non-stationary (p > 0.05)
        if p1 > 0.05 and p2 > 0.05:
            p_cointegration1 = coint(series1, series2, trend='c')[1]  # Engle-Granger test p-value
            p_cointegration2 = coint(series2, series1, trend='c')[1]
        else:
            p_cointegration1 = np.nan  # Not applicable
            p_cointegration2 = np.nan

        results.append((t1, t2, p1, p2, p_cointegration1, p_cointegration2))

    # Convert results into DataFrame and merge with original correlation DataFrame
    results_df = pd.DataFrame(results, columns=["ticker_1", "ticker_2", "p1_stationary", "p2_stationary", "p_cointegration1", "p_cointegration2"])
    return corr_df.merge(results_df, on=["ticker_1", "ticker_2"])

corr_df = calculate_cointegration(so, df)
corr_df.to_csv('Data/SnP500cointegration_results_temp2.csv')
corr_df['p_coint_avg'] = corr_df[['p_cointegration1', 'p_cointegration2']].mean(axis=1, skipna=True)
corr_df['p_coint_min'] = corr_df[['p_cointegration1', 'p_cointegration2']].min(axis=1, skipna=True)
corr_df['p_coint_max'] = corr_df[['p_cointegration1', 'p_cointegration2']].max(axis=1, skipna=True)
corr_df.to_csv('Data/SnP500cointegration_results_202001_202501.csv')

In [ ]:
promising_pairs1 = corr_df[(corr_df['p_coint_max']>0) &
        (corr_df['p_coint_max']<0.01) &
        (corr_df['p1_stationary']>=0.05) &
        (corr_df['p2_stationary']>=0.05) &
        (~corr_df['correlation'].isna())
        ].sort_values(by='p_coint_max', ascending=True)

In [ ]:
promising_pairs1['regress1on2'] = promising_pairs1['p_cointegration1'] < promising_pairs1['p_cointegration2']

In [ ]:
promising_pairs1

In [ ]:
def plot_dual_axis(df, ticker1, ticker2):
    """
    Plots two time series on separate y-axes.

    Parameters:
        df (pd.DataFrame): DataFrame containing the time series.
        ticker1 (str): First ticker (plotted on left y-axis).
        ticker2 (str): Second ticker (plotted on right y-axis).
    """
    fig, ax1 = plt.subplots(figsize=(10, 5))

    # First time series (ticker1)
    ax1.plot(df.index, df[ticker1], marker=".", markersize=1, color="b", label=ticker1)
    ax1.set_xlabel("Date")
    ax1.set_ylabel(f"{ticker1} Value", color="b")
    ax1.tick_params(axis="y", labelcolor="b")
    ax1.grid(True, linestyle="--", alpha=0.5)

    # Second time series (ticker2) on secondary y-axis
    ax2 = ax1.twinx()
    ax2.plot(df.index, df[ticker2], marker=".", markersize=1, color="r", label=ticker2)
    ax2.set_ylabel(f"{ticker2} Value", color="r")
    ax2.tick_params(axis="y", labelcolor="r")

    # Legend
    fig.legend(loc="upper left", bbox_to_anchor=(0.1, 0.9))
    plt.title(f"{ticker1} vs {ticker2} Time Series")

    # Show plot
    plt.show()

In [ ]:
def plot_same_axis(df, ticker1, ticker2, factor1=1, factor2=1):
    """
    Plots two time series on the same y-axis.

    Parameters:
        df (pd.DataFrame): DataFrame containing the time series.
        ticker1 (str): First ticker to plot.
        ticker2 (str): Second ticker to plot.
        factor1 (float, optional): Multiplicative factor for the first series. Default is 1.
        factor2 (float, optional): Multiplicative factor for the second series. Default is 1.
    """
    fig, ax = plt.subplots(figsize=(10, 5))

    # Plot both series on the same axis
    ax.plot(df.index, df[ticker1] * factor1, marker=".", markersize=1, color="b", label=ticker1)
    ax.plot(df.index, df[ticker2] * factor2, marker=".", markersize=1, color="r", label=ticker2)

    # Labels and title
    ax.set_xlabel("Date")
    ax.set_ylabel("Value")
    ax.set_title(f"{ticker1} vs {ticker2} Time Series")
    ax.legend()
    ax.grid(True, linestyle="--", alpha=0.5)

    # Show plot
    plt.show()

In [ ]:
plot_dual_axis(df, "ADP", "BKR")  # Pass any two tickers


In [ ]:
def get_cointegration_factor(ticker_1_series, ticker_2_series, f_1on2):
    """
    Calculates the cointegration factor (β) using OLS regression without an intercept.

    Parameters:
        df (pd.DataFrame): DataFrame containing time series data.
        ticker_1 (str): Independent variable.
        ticker_2 (str): Dependent variable.
        f_1on2 (bool): Determines if its series1~series2 or series2~series1

    Returns:
        float: Cointegration factor (β), or NaN if computation fails.
    """
    try:
        series1 = ticker_1_series.dropna()
        series2 = ticker_2_series.dropna()

        # Ensure both series have the same length
        min_len = min(len(series1), len(series2))
        if min_len < 20:
            return np.nan  # Not enough data points

        series1, series2 = series1.iloc[-min_len:], series2.iloc[-min_len:]
        if (f_1on2):
            # OLS Regression: ticker_1 ~ ticker_2 (without intercept)
            model = sm.OLS(series1, series2, hasconst=True).fit()
        else:
            model = sm.OLS(series2, series1, hasconst=True).fit()
        return model.params[0]  # Extract β coefficient (slope)

    except Exception as e:
        print("Exception occurred ",e )
        return np.nan  # Return NaN if an error occurs

In [ ]:
promising_pairs1['coint_factor'] = promising_pairs1.apply(lambda row: get_cointegration_factor(df[row['ticker_1']], df[row['ticker_2']], row['regress1on2']), axis=1)


In [ ]:
promising_pairs1.to_csv('Data/promising_pairs1.csv', index=False)

In [ ]:
promising_pairs1.head()

In [ ]:
plot_dual_axis(df, "ADP", "BKR")

In [ ]:
get_cointegration_factor(df['ADP'], df['BKR'],True)

In [ ]:
plot_same_axis(df, "ADP", "BKR",0.12744798758111703, 1)

In [ ]:
def calculate_residuals(series_y, series_x, beta):


    residuals = series_y / (beta * series_x) - 1
    mean = residuals.mean()
    std = residuals.std()
    residuals_std = (residuals - mean) / std
    return residuals, residuals_std, mean, std

In [ ]:
promising_pairs1[['coint_resid_mean', 'coint_resid_std']] = promising_pairs1.apply(
    lambda row: pd.Series(
        calculate_residuals(df[row['ticker_1']], df[row['ticker_2']], row['coint_factor'])[2:4]
        if row['regress1on2']
        else calculate_residuals(df[row['ticker_2']], df[row['ticker_1']], row['coint_factor'])[2:4]
    ),
    axis=1
)


In [ ]:
promising_pairs1['current']

In [ ]:
def get_latest_resid(symbol_y, symbol_x, beta, mean, std):
    latest_y = yf.Ticker(symbol_y).fast_info['lastPrice']
    latest_x = yf.Ticker(symbol_x).fast_info['lastPrice']
    latest_resid = latest_y / (beta * latest_x) - 1
    return (latest_resid - mean)/std

promising_pairs1['current_signal'] = promising_pairs1.apply(lambda row: get_latest_resid(row['ticker_1'], row['ticker_2'], row['coint_factor'], row['coint_resid_mean'], row['coint_resid_std']) if row['regress1on2'] else get_latest_resid(row['ticker_2'], row['ticker_1'], row['coint_factor'], row['coint_resid_mean'], row['coint_resid_std']), axis=1)


In [ ]:
promising_pairs1[promising_pairs1['current_signal'].abs() > 2.5]

In [ ]:
calculate_residuals(df['MO'], df['PM'],2.2562831761575657)[1].plot()

In [ ]:
plot_same_axis(df, "MO", "PM",2.2562831761575657,1)

In [ ]:
ticker_1, factor_1 = 'MO',2.2562831761575657
ticker_2, factor_2 = 'PM',1


ticker_1_latest = yf.Ticker(ticker_1).fast_info['lastPrice']
ticker_2_latest = yf.Ticker(ticker_2).fast_info['lastPrice']
midpoint = (ticker_1_latest*factor_1 + ticker_2_latest*factor_2)/2

print(f"ticker {ticker_1} latest price is {ticker_1_latest}")
print(f"ticker {ticker_2} latest price is {ticker_2_latest}")
print(f"Average middle is {midpoint}, which is {midpoint/factor_1} for {ticker_1} and {midpoint/factor_2} for {ticker_2}")



In [ ]:
df['AVGO']

In [ ]:
#download live latest data
ticker_1 = 'JPM'
ticker_2 = 'RL'
ticker_1_live = yf.Ticker(ticker_1).history(interval = '1d', period='1y')[['Close']].rename(columns={'Close':f'close_{ticker_1}'})
ticker_2_live = yf.Ticker(ticker_2).history(interval = '1d', period='1y')[['Close']].rename(columns={'Close':f'close_{ticker_2}'})
ticker = ticker_1_live.merge(ticker_2_live, left_index=True, right_index=True, how='outer')

plot_same_axis(ticker, f'close_{ticker_1}', f'close_{ticker_2}',1,1.174558)

In [ ]:
plot_same_axis(df, "MO", "PM",2.2562831761575657,1)

In [ ]:
yf.Ticker('S68.SI').news